In [1]:
!pip install gmaps

In [2]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [3]:
!jupyter nbextension enable --py --sys-prefix gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


In [5]:
weather = "Weather.csv"
weather_df = pd.read_csv(weather)
weather_df.head()

,City,Country,Cloudiness,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed
0,Ostrovnoy,RU,100,1587156261,96,68.05,39.51,32.29,14.74
1,Mar del Plata,AR,65,1587156139,52,-38.00,-57.56,71.01,11.41
2,Arraial do Cabo,BR,75,1587156261,78,-22.97,-42.03,73.40,10.29
3,Beringovskiy,RU,99,1587156174,96,63.05,179.32,19.31,22.03
4,Port Alfred,ZA,11,1587156261,91,-33.59,26.89,64.00,4.00


In [6]:
gmaps.configure(api_key=g_key)

In [7]:
locations = weather_df[["Latitude", "Longitude"]]
humidity = weather_df["Humidity"].astype(float)
max_intensity = weather_df["Humidity"].max()

In [8]:
fig = gmaps.figure(center=(25,0), zoom_level=1.80)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, max_intensity=max_intensity)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
ideal_df = weather_df.loc[(weather_df["Max Temperature"] > 60) & (weather_df["Max Temperature"] < 85) & (weather_df["Wind Speed"] < 3) & (weather_df["Cloudiness"] < 15 )]
ideal_df.dropna
ideal_df.count()

City               7
Country            7
Cloudiness         7
Date               7
Humidity           7
Latitude           7
Longitude          7
Max Temperature    7
Wind Speed         7
dtype: int64

In [10]:
hotel_df = ideal_df.copy()
hotel_df = hotel_df.reset_index()
del hotel_df["index"]
hotel_df["Hotel Name"] = ""


In [11]:
latitude = hotel_df["Latitude"]
longitude = hotel_df["Longitude"]
length_i = len(hotel_df)

In [12]:
for i in range(length_i):
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    params ={
        "location": f'{latitude[i]}, {longitude[i]}',
        "radius": 5000,
        "types": "lodging",
        "keyword": "Hotel",
        "key": g_key
    }
    
    response = requests.get(base_url, params = params).json()
    
    try:
        results = response["results"][i]["name"]
        print(f'Nearest Lodging: {results}')
        hotel_df.loc[hotel_df["City"] == hotel_df["City"][i], "Hotel Name"] = results
    except IndexError:
        print("No Lodging within 5000 meters")

Nearest Lodging: Jupiter Hotel Portland
Nearest Lodging: Pension Bianca & Benoit
Nearest Lodging: AMBER PALACE HOTEL
Nearest Lodging: Apart Hotel Demetrio
No Lodging within 5000 meters
Nearest Lodging: Pensao Cruzeiro
No Lodging within 5000 meters


In [13]:
hotel_df = hotel_df[hotel_df["Hotel Name"] != '']
hotel_df = hotel_df.reset_index()
del hotel_df["index"]
hotel_df

,City,Country,Cloudiness,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed,Hotel Name
0,Portland,US,0,1587155700,29,45.52,-122.68,73.99,2.24,Jupiter Hotel Portland
1,Rikitea,PF,1,1587156277,65,-23.12,-134.97,77.18,2.51,Pension Bianca & Benoit
2,Myitkyina,MM,0,1587156286,78,25.38,97.40,67.37,2.33,AMBER PALACE HOTEL
3,Olavarría,AR,0,1587156292,47,-36.89,-60.32,73.00,1.99,Apart Hotel Demetrio
4,Mocuba,MZ,4,1587156308,93,-16.84,36.99,70.90,2.17,Pensao Cruzeiro


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [15]:
markers = gmaps.marker_layer(locations, info_box_content= hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))